<a href="https://colab.research.google.com/github/cristobalperezp/Marketing_2/blob/main/Class_09_Continuous_Time_Duration_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Class_09_Continuous_Time_Duration_Models

In [ ]:
#librerías para trabajo con dataframes:
import pandas as pd
import numpy as np
#librerías para graficar:
import seaborn as sns
import matplotlib.pyplot as plt
#optimizador:
from scipy.optimize import minimize
#librerías de distribuciones:
import scipy.stats as stats
import scipy.special as special

In [ ]:
#cargamos las bases de datos:
!git clone https://github.com/cristobalperezp/Marketing_II.git

fatal: destination path 'Marketing_II' already exists and is not an empty directory.


In [ ]:
#se lee la base de datos
mydata = pd.read_csv('/content/Marketing_II/krunchy bits data.csv')
mydata

,Week,Actual,IncrHH
0,1,8,8
1,2,14,6
2,3,16,2
3,4,32,16
4,5,40,8
5,6,47,7
6,7,50,3
7,8,52,2
8,9,57,5
9,10,60,3


In [ ]:
train = mydata.iloc[0:24]
test = mydata.iloc[25:]
nc = train.shape[0]
nv = test.shape[0]

nt = nc + nv  # total number of periods
n = 1499     # total number households

In [ ]:
#(1) gamma exponential model --------------------------------------------------------
def loglge(theta):
  #transform variable to impose is in the (0,1) range
  myr = np.exp(theta[0])
  myalpha = np.exp(theta[1])
  #define auxiliary arrays
  pr = np.zeros(nc+1)
  ll = np.zeros(nc+1)
  #detection likelihood
  pr[0] = 1-(myalpha/(myalpha+train['Week'][0]))**myr
  ll[0] = train['IncrHH'][0]*np.log(pr[0])
  for i in range(1,nc):
    pr[i] = 1-(myalpha/(myalpha+train['Week'][i]))**myr
    ll[i] = train['IncrHH'][i]*np.log(pr[i] - pr[i-1])
  #survival likelihood
  ll[nc] = (n-train['Actual'][nc-1])*np.log(1-pr[nc-1])
  return -np.sum(ll)

In [ ]:
theta_start = [0.6666,0.6666]

#Run the minimizer
results = minimize(loglge,theta_start, method = 'BFGS',options={'maxiter': 30000, 'disp': True})

#Print the results
myr = np.exp(results.x[0])
myalpha = np.exp(results.x[1])
myr,myalpha

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


         Current function value: nan
         Iterations: 2
         Function evaluations: 452
         Gradient evaluations: 113


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-pa

(inf, inf)

In [ ]:
#(1) gamma weibull model --------------------------------------------------------
def loglgw(theta):
  #transform variable to impose they are positive
  myr = np.exp(theta[0])
  myalpha = np.exp(theta[1])
  myc = np.exp(theta[2])
  #define auxiliary arrays
  pr = np.zeros(nc+1)
  ll = np.zeros(nc+1)
  #detection likelihood
  pr[0] = 1-(myalpha/(myalpha+train['Week'][0]**myc))**myr
  ll[0] = train['IncrHH'][0]*np.log(pr[0])
  for i in range(1,nc):
    pr[i] = 1-(myalpha/(myalpha+train['Week'][i]**myc))**myr
    ll[i] = train['IncrHH'][i]*np.log(pr[i] - pr[i-1])
  #survival likelihood
  ll[nc] = (n-train['Actual'][nc-1])*np.log(1-pr[nc-1])
  return -np.sum(ll)

In [ ]:
theta_start = [0.6666,0.6666,0.8888]

#Run the minimizer
results = minimize(loglgw,theta_start, method = 'BFGS',options={'maxiter': 30000, 'disp': True})

#Print the results
myr = np.exp(results.x[0])
myalpha = np.exp(results.x[1])
myc = np.exp(results.x[2])
myr,myalpha,myc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


         Current function value: nan
         Iterations: 2
         Function evaluations: 565
         Gradient evaluations: 113


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/

(inf, inf, 0.0)